In [ ]:
number = 23
input_filename = "username_split" + str(number) + ".txt"
output_filename = "split" + str(number) + "_result.csv"

import time
import datetime
import requests
import csv
from tqdm import notebook

session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

error_url = []
21
def FetchDataFromGitHubRestApi(url):
    response = session.get(url)
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    elif response.status_code == 403:
        until_next_hour = 60 - datetime.datetime.now().minute
        for x in notebook.tqdm(range(until_next_hour)):
            time.sleep(60)
        response = session.get(url)
        if response.status_code == 200:
            fetched_data = response.json()
            poll_interval = None
            poll_interval = response.headers.get('X-Poll-Interval')
            if poll_interval:
                time.sleep(int(poll_interval))
            return fetched_data
        else:
            error_url.append(url)
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output
    
def CollectUserEventData(event_url):

    all_event_data = []
    fetched_event_data = FetchDataFromGitHubRestApi(event_url)

    for event in fetched_event_data:
        try:
            type_ = Select("type", event)
            created_at = Select("created_at", event)
            all_event_data.append([type_, created_at])
        except:
            pass

    return all_event_data

with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

with open ("not_a_user_or_no_data.txt", "r", encoding="utf8") as f:
    not_a_user_or_no_data = f.readlines()

for user in not_a_user_or_no_data:
    all_username.remove(user)

all_username = [x.strip() for x in all_username]
user_alias = ["usr"+str(c) for c in range(len(all_username))]

username_dict = {}
for i in range(len(user_alias)):
    username_dict[all_username[i]] = user_alias[i]

with open (input_filename, "r", encoding="utf8") as f:
    username_from_split_dataset = f.readlines()

username_from_split_dataset = [x.strip() for x in username_from_split_dataset]

print("start time: ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

with open(output_filename, 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "event_type", "created_at"])

    for username in notebook.tqdm(username_from_split_dataset):
        event_url = "https://api.github.com/users/"+username+"/events?per_page=100"
        page1 = []
        page2 = []
        page3 = []
        try:
            page1 = CollectUserEventData(event_url+"&page=1")
            if len(page1) == 100:
                page2 = CollectUserEventData(event_url+"&page=2")
                for event in page2:
                    page1.append(event)
                if len(page2) == 100:
                    page3 = CollectUserEventData(event_url+"&page=3")
                    for event in page3:
                        page1.append(event)
            if page1:
                for event in page1:
                    writer.writerow([username_dict[username], event[0], event[1]])
            else:
                writer.writerow([username_dict[username], "None", "None"])
        except:
            writer.writerow([username_dict[username], "None", "None"])
print("done!")
print("stop time: ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

start time:  12 : 9


  0%|          | 0/209 [00:00<?, ?it/s]